In [4]:
import json
import pennylane as qml
import pennylane.numpy as np

wires_m = [0, 1, 2]  # qubits needed to encode m
wires_n = [3, 4, 5]  # qubits needed to encode n
wires_aux = [6, 7, 8, 9, 10]  # auxiliary qubits you can use


# Put your code here #

# Create all the helper functions you need here
def sub_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(-k * np.pi / (2**j), wires=wires[j])

def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

def quantum_OR(wire1, wire2, target):
    # Apply CNOT gates to propagate the OR condition to the target qubit
    qml.CNOT(wires=[wire1, target])
    qml.CNOT(wires=[wire2, target])

    # Use a Toffoli gate to reset the target if both wire1 and wire2 were 1
    # This step assumes that the target qubit is initially in the |0> state
    qml.Toffoli(wires=[wire1, wire2, target])

def reverse_quantum_OR(wire1, wire2, target):
    # Apply Toffoli gate to reset the target if both wire1 and wire2 were 1
    qml.Toffoli(wires=[wire1, wire2, target])
    
    # Apply CNOT gates to undo the OR condition on the target qubit
    qml.CNOT(wires=[wire2, target])
    qml.CNOT(wires=[wire1, target])



def oracle_distance(d):
    """
    Args:
        d (int): the distance with which we will check that the oracle is working properly.

    This function does not return anything, it is a quantum function that applies
    necessary gates that implement the requested oracle.

    """


    # Put your code here

    # There is an error in pennylane 0.33. If a qubit is not used, the output is of the state is a numpy array so the test case fails without the following line :
    for i in range(11):
        qml.Identity(wires=i)


    # We divid the equation into two parts : n+d-m = 0 and m+d-n = 0

    # 0 = n+d-m
    n_plus_one_reg = [wires_aux[0]] + wires_n
    qml.QFT(wires=n_plus_one_reg)
    add_k_fourier(d, n_plus_one_reg)
    for i in range(len(wires_m)):
        qml.ctrl(sub_k_fourier, control=wires_m[i])(2 **(len(wires_m) - i - 1), n_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=n_plus_one_reg)

    # If everything is zero, set the first qubit to 1
    qml.ctrl(qml.PauliX, control=n_plus_one_reg, control_values=(0,0,0,0))(wires_aux[1])
    
    qml.QFT(wires=n_plus_one_reg)
    add_k_fourier(-d, n_plus_one_reg)
    for i in range(len(wires_m)):
        qml.ctrl(add_k_fourier, control=wires_m[i])(2 **(len(wires_m) - i - 1), n_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=n_plus_one_reg)


    # 0 = m+d-n
    m_plus_one_reg = [wires_aux[0]] + wires_m
    qml.QFT(wires=m_plus_one_reg)
    add_k_fourier(d, m_plus_one_reg)
    for i in range(len(wires_n)):
        qml.ctrl(sub_k_fourier, control=wires_n[i])(2 **(len(wires_n) - i - 1), m_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=m_plus_one_reg)

    # If everything is zero, set the first qubit to 1
    qml.ctrl(qml.PauliX, control=m_plus_one_reg, control_values=(0,0,0,0))(wires_aux[2])
    
    qml.QFT(wires=m_plus_one_reg)
    add_k_fourier(-d, m_plus_one_reg)
    for i in range(len(wires_n)):
        qml.ctrl(add_k_fourier, control=wires_n[i])(2 **(len(wires_n) - i - 1), m_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=m_plus_one_reg)
    
    #################################################################################################    
    # Set 1 state if the distance is correct
    quantum_OR(wires_aux[1], wires_aux[2], wires_aux[3])
    qml.PauliZ(wires=wires_aux[3])
    reverse_quantum_OR(wires_aux[1], wires_aux[2], wires_aux[3])
    #################################################################################################
    
    # 0 = n+d-m
    qml.QFT(wires=m_plus_one_reg)
    add_k_fourier(-d, m_plus_one_reg)
    for i in range(len(wires_n)):
        qml.ctrl(add_k_fourier, control=wires_n[i])(2 **(len(wires_n) - i - 1), m_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=m_plus_one_reg)

    # If everything is zero, set the first qubit to 1
    qml.ctrl(qml.PauliX, control=m_plus_one_reg, control_values=(0,0,0,0))(wires_aux[2])

    m_plus_one_reg = [wires_aux[0]] + wires_m
    qml.QFT(wires=m_plus_one_reg)
    add_k_fourier(d, m_plus_one_reg)
    for i in range(len(wires_n)):
        qml.ctrl(sub_k_fourier, control=wires_n[i])(2 **(len(wires_n) - i - 1), m_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=m_plus_one_reg)
    
    # 0 = n+d-m
    n_plus_one_reg = [wires_aux[0]] + wires_n
    qml.QFT(wires=n_plus_one_reg)
    add_k_fourier(d, n_plus_one_reg)
    for i in range(len(wires_m)):
        qml.ctrl(sub_k_fourier, control=wires_m[i])(2 **(len(wires_m) - i - 1), n_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=n_plus_one_reg)

    # If everything is zero, set the first qubit to 1
    qml.ctrl(qml.PauliX, control=n_plus_one_reg, control_values=(0,0,0,0))(wires_aux[1])
    
    qml.QFT(wires=n_plus_one_reg)
    add_k_fourier(-d, n_plus_one_reg)
    for i in range(len(wires_m)):
        qml.ctrl(add_k_fourier, control=wires_m[i])(2 **(len(wires_m) - i - 1), n_plus_one_reg)
    qml.adjoint(qml.QFT)(wires=n_plus_one_reg)



In [5]:
dev = qml.device("default.qubit", wires=11)

@qml.qnode(dev)
def circuit(m, n, d):
    qml.BasisEmbedding(m, wires=wires_m)
    qml.BasisEmbedding(n, wires=wires_n)
    oracle_distance(d)
    return qml.state()


def run(test_case_input: str) -> str:
    outputs = []
    d = int(json.loads(test_case_input))
    for n in range(8):
        for m in range(8):
            outputs.append(sum(circuit(n, m, d)).real)
    outputs.append(d)
    output_list = [elem.numpy() for elem in outputs[:-1]] + [outputs[-1]]
    return str(output_list)


def check(solution_output: str, expected_output: str) -> None:
    i = 0
    solution_output = json.loads(solution_output)
    d = solution_output[-1]
    assert expected_output == "No output", "Something went wrong"
    for n in range(8):
        for m in range(8):
            solution = 1
            if abs(n - m) == d:
                solution = -1
            assert np.isclose(solution_output[i], solution)
            i += 1

    circuit(np.random.randint(7), np.random.randint(7), np.random.randint(7))
    tape = circuit.qtape

    names = [op.name for op in tape.operations]

    assert names.count("QubitUnitary") == 0, "Can't use custom-built gates!"


# These are the public test cases
test_cases = [
    ('0', 'No output'),
    ('1', 'No output'),
    ('2', 'No output'),
    ('3', 'No output'),
    ('4', 'No output'),
    ('5', 'No output'),
    ('6', 'No output'),
    ('7', 'No output')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '0'...
Correct!
Running test case 1 with input '1'...
Correct!
Running test case 2 with input '2'...
Correct!
Running test case 3 with input '3'...
Correct!
Running test case 4 with input '4'...
Correct!
Running test case 5 with input '5'...
Correct!
Running test case 6 with input '6'...
Correct!
Running test case 7 with input '7'...
Correct!
